In [1]:
__author__ = "Tim Cuddeback"

import yfpy, json, ast
from yfpy.query import YahooFantasySportsQuery
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
# Function that pulls data from Yahoo's Fantasy API and returns a sorted list of tuples with (team_id,current points)
league_id = '12514' # put your real league id here
game_id = "399" # put the game id here (game id's reflect the type of sport and the year)
game_code = "nfl" # put the game code here
season = "2020" # put the year of the current fantasy season
auth_dir = '' # put the location where you are storing the client_id/secret
working_dir = '' # put the location where you want excel file to be saved
excel_file_name = 'Points.xlsx' # Name your file after you set up the headers

# Query yffpy to get data
yahoo_query = YahooFantasySportsQuery(auth_dir, league_id, game_id=game_id, game_code=game_code, offline=False)

# Get standings data to get the points for the day
data = yahoo_query.get_league_matchups_by_week(1)
# .get_league_teams(self)
# .get_league_players(self)
# .get_league_draft_results(self)
# .get_league_matchups_by_week(self,chosen_week)
# .get_team_info(self,team_id)
# .get_team_standings(self,team_id) # has current "streak" within
# .get_player_stats_by_week(self, player_key, chosen_week="current")

[2021-04-10 19:26:13,128 DEBUG] [yfpy.query.query.query] Response (JSON): {'fantasy_content': {'xml:lang': 'en-US', 'yahoo:uri': '/fantasy/v2/game/399/metadata', 'game': [{'game_key': '399', 'game_id': '399', 'name': 'Football', 'code': 'nfl', 'type': 'full', 'url': 'https://football.fantasysports.yahoo.com/f1', 'season': '2020', 'is_registration_over': 1, 'is_game_over': 1, 'is_offseason': 1}], 'time': '16.286134719849ms', 'copyright': 'Data provided by Yahoo! and STATS, LLC', 'refresh_rate': '60'}}
[2021-04-10 19:26:13,134 DEBUG] [yfpy.query.query.query] Data fetched with query URL: https://fantasysports.yahooapis.com/fantasy/v2/game/399/metadata?format=json
[2021-04-10 19:26:13,135 DEBUG] [yfpy.query.query.query] Response (Yahoo fantasy data extracted from: "fantasy_content"): {'xml:lang': 'en-US', 'yahoo:uri': '/fantasy/v2/game/399/metadata', 'game': [{'game_key': '399', 'game_id': '399', 'name': 'Football', 'code': 'nfl', 'type': 'full', 'url': 'https://football.fantasysports.yaho

In [3]:
# df.row.str.extract()
df = pd.DataFrame(data[0]['matchup'])

In [4]:
df

,0
0,1
1,2020-09-10
2,2020-09-14
3,postevent
4,0
5,0
6,1
7,https://football.fantasysports.yahoo.com/2020/f1/12514/recap?week=1&mid1=1&mid2=2
8,"Fantasy Sidelines smirk at predictions, defeat FreshPrinceOfHelaire, 110.1-101.6"
9,"[{'matchup_grade': { ""grade"": ""B+"", ""team_key"": ""399.l.12514.t.1"" }}, {'matchup_grade': { ""grade"": ""B"", ""team_key"": ""399.l.12514.t.2"" }}]"
